In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile7 = pd.read_pickle("FFF_Train_Decile7.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile7.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile7, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 7')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 5s 2ms/step - loss: 0.3273 - acc: 0.8727 - val_loss: 0.2968 - val_acc: 0.8887
Epoch 39/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3264 - acc: 0.8694 - val_loss: 0.2928 - val_acc: 0.8887
Epoch 40/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3222 - acc: 0.8700 - val_loss: 0.2916 - val_acc: 0.8848
Epoch 41/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3210 - acc: 0.8687 - val_loss: 0.2906 - val_acc: 0.8874
Epoch 42/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3247 - acc: 0.8740 - val_loss: 0.3003 - val_acc: 0.8835
Epoch 43/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3212 - acc: 0.8740 - val_loss: 0.2931 - val_acc: 0.8874
Epoch 44/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3213 - acc: 0.8730 - val_loss: 0.2920 - val_acc: 0.8927
Epoch 45/100
3055/3055 [==============================] - 5s 

Epoch 98/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3139 - acc: 0.8756 - val_loss: 0.2877 - val_acc: 0.8901
Epoch 99/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3131 - acc: 0.8773 - val_loss: 0.2878 - val_acc: 0.8901
Epoch 100/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3166 - acc: 0.8795 - val_loss: 0.2883 - val_acc: 0.8927
Test accuracy: 0.8926701561318642
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.5832 - acc: 0.6668 - val_loss: 0.4472 - val_acc: 0.8691
Epoch 2/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4328 - acc: 0.8547 - val_loss: 0.3690 - val_acc: 0.8691
Epoch 3/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3877 - acc: 0.8622 - val_loss: 0.3222 - val_acc: 0.8796
Epoch 4/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3580 - acc: 0.8602 - val_l

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2819 - acc: 0.8799 - val_loss: 0.2614 - val_acc: 0.9058
Epoch 57/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2758 - acc: 0.8864 - val_loss: 0.2610 - val_acc: 0.9058
Epoch 58/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2825 - acc: 0.8851 - val_loss: 0.2609 - val_acc: 0.9071
Epoch 59/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2771 - acc: 0.8845 - val_loss: 0.2609 - val_acc: 0.9071
Epoch 60/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2839 - acc: 0.8848 - val_loss: 0.2609 - val_acc: 0.9071
Epoch 61/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2750 - acc: 0.8831 - val_loss: 0.2609 - val_acc: 0.9071
Epoch 62/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2797 - acc: 0.8838 - val_loss: 0.2608 - val_acc: 0.9071
Epoch 63/150
3055/3055 [==============================] - 5s 

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2787 - acc: 0.8881 - val_loss: 0.2595 - val_acc: 0.9058
Epoch 117/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2735 - acc: 0.8877 - val_loss: 0.2595 - val_acc: 0.9045
Epoch 118/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2767 - acc: 0.8874 - val_loss: 0.2594 - val_acc: 0.9045
Epoch 119/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2809 - acc: 0.8900 - val_loss: 0.2595 - val_acc: 0.9045
Epoch 120/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2791 - acc: 0.8864 - val_loss: 0.2595 - val_acc: 0.9058
Epoch 121/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2792 - acc: 0.8845 - val_loss: 0.2596 - val_acc: 0.9031
Epoch 122/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2764 - acc: 0.8887 - val_loss: 0.2590 - val_acc: 0.9045
Epoch 123/150
3055/3055 [==============================

...

Epoch 67/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3156 - acc: 0.8750 - val_loss: 0.2900 - val_acc: 0.8822
Epoch 68/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3206 - acc: 0.8750 - val_loss: 0.2905 - val_acc: 0.8835
Epoch 69/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3176 - acc: 0.8746 - val_loss: 0.2899 - val_acc: 0.8822
Epoch 70/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3178 - acc: 0.8759 - val_loss: 0.2899 - val_acc: 0.8848
Epoch 71/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3200 - acc: 0.8746 - val_loss: 0.2905 - val_acc: 0.8809
Epoch 72/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3196 - acc: 0.8736 - val_loss: 0.2898 - val_acc: 0.8822
Epoch 73/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3123 - acc: 0.8720 - val_loss: 0.2894 - val_acc: 0.8822
Epoch 74/100
3055/3055 [========================

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2787 - acc: 0.8845 - val_loss: 0.2648 - val_acc: 0.9005
Epoch 27/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2828 - acc: 0.8874 - val_loss: 0.2636 - val_acc: 0.9018
Epoch 28/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2800 - acc: 0.8802 - val_loss: 0.2639 - val_acc: 0.9005
Epoch 29/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2777 - acc: 0.8858 - val_loss: 0.2652 - val_acc: 0.8953
Epoch 30/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2813 - acc: 0.8841 - val_loss: 0.2660 - val_acc: 0.8966
Epoch 31/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2826 - acc: 0.8805 - val_loss: 0.2652 - val_acc: 0.8992
Epoch 32/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2877 - acc: 0.8831 - val_loss: 0.2657 - val_acc: 0.8992

Epoch 00032: ReduceLROnPlateau reducing learning rate to 1.0

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2782 - acc: 0.8894 - val_loss: 0.2657 - val_acc: 0.8979
Epoch 86/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2763 - acc: 0.8861 - val_loss: 0.2657 - val_acc: 0.8979
Epoch 87/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2758 - acc: 0.8861 - val_loss: 0.2658 - val_acc: 0.8979
Epoch 88/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2804 - acc: 0.8858 - val_loss: 0.2657 - val_acc: 0.8979
Epoch 89/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2788 - acc: 0.8845 - val_loss: 0.2658 - val_acc: 0.8992
Epoch 90/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2789 - acc: 0.8871 - val_loss: 0.2658 - val_acc: 0.8979
Epoch 91/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2801 - acc: 0.8890 - val_loss: 0.2659 - val_acc: 0.8966
Epoch 92/200
3055/3055 [==============================] - 6s 

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2800 - acc: 0.8887 - val_loss: 0.2662 - val_acc: 0.8979
Epoch 146/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2786 - acc: 0.8887 - val_loss: 0.2663 - val_acc: 0.8979
Epoch 147/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2753 - acc: 0.8884 - val_loss: 0.2662 - val_acc: 0.8979
Epoch 148/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2787 - acc: 0.8841 - val_loss: 0.2661 - val_acc: 0.8979
Epoch 149/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2805 - acc: 0.8871 - val_loss: 0.2661 - val_acc: 0.8979
Epoch 150/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2763 - acc: 0.8838 - val_loss: 0.2661 - val_acc: 0.8992
Epoch 151/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2798 - acc: 0.8845 - val_loss: 0.2662 - val_acc: 0.8979
Epoch 152/200
3055/3055 [==============================

3055/3055 [==============================] - 5s 2ms/step - loss: 0.4138 - acc: 0.8484 - val_loss: 0.3613 - val_acc: 0.8626
Epoch 5/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3709 - acc: 0.8609 - val_loss: 0.3172 - val_acc: 0.8835
Epoch 6/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3535 - acc: 0.8645 - val_loss: 0.3079 - val_acc: 0.8887
Epoch 7/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3439 - acc: 0.8664 - val_loss: 0.3039 - val_acc: 0.8848
Epoch 8/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3478 - acc: 0.8602 - val_loss: 0.3083 - val_acc: 0.8861
Epoch 9/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3352 - acc: 0.8704 - val_loss: 0.3050 - val_acc: 0.8874
Epoch 10/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3250 - acc: 0.8710 - val_loss: 0.3041 - val_acc: 0.8914
Epoch 11/50
3055/3055 [==============================] - 5s 2ms/step - l

In [ ]:
0.93484

In [ ]:
#0.90